In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install torchxrayvision
!pip install evaluate
!pip install rouge_score

In [ ]:
from unsloth import FastLanguageModel
import torch
from transformers import AutoModelForCausalLM, AutoModel, AutoProcessor, AutoTokenizer, TrainingArguments, EarlyStoppingCallback
import pandas as pd
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
import torchxrayvision as xrv
import numpy as np
import zipfile
import os
from datasets import Dataset, DatasetDict

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

import pandas as pd
from evaluate import load
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score

import locale
locale.getpreferredencoding = lambda: "UTF-8"

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# from google.colab import drive
# import shutil

# # Mount Google Drive
# drive.mount('/content/drive')

# # Set the path to your folder in Google Drive
# drive_folder_path = '/content/drive/MyDrive/my_model_2'

# # Set the destination path in Colab
# colab_folder_path = '/content/my_model'

# # Copy the folder from Google Drive to Colab environment
# shutil.copytree(drive_folder_path, colab_folder_path)

# # Verify contents in the Colab environment
# import os
# files = os.listdir(colab_folder_path)
# print("Files in Colab folder:", files)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in Colab folder: ['README.md', 'adapter_model.safetensors', 'tokenizer.json', 'adapter_config.json', 'special_tokens_map.json', 'tokenizer_config.json']


In [ ]:
max_seq_length = 300 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
df = pd.read_csv("final.csv")

In [ ]:
df.head()

,f,image_names,embedding,report,instruction
0,1,"['1_IM-0001-4001.dcm.png', '1_IM-0001-3001.dcm...","[0.014, 0.137, 0.119, 0.023, 0.0, 0.018, 0.187...",The cardiac silhouette and mediastinum size ar...,Generate a medical report based on the given c...
1,2,"['2_IM-0652-1001.dcm.png', '2_IM-0652-2001.dcm...","[0.395, 0.132, 0.346, 0.102, 0.121, 0.17, 0.50...",Borderline cardiomegaly. Midline sternotomy En...,Generate a medical report based on the given c...
2,4,"['4_IM-2050-1001.dcm.png', '4_IM-2050-2001.dcm...","[0.528, 0.658, 0.598, 0.536, 0.215, 0.505, 0.5...",There are diffuse bilateral interstitial and a...,Generate a medical report based on the given c...
3,5,"['5_IM-2117-1003002.dcm.png', '5_IM-2117-10040...","[0.53, 0.302, 0.508, 0.037, 0.007, 0.353, 0.51...",The cardiomediastinal silhouette and pulmonary...,Generate a medical report based on the given c...
4,6,"['6_IM-2192-1001.dcm.png', '6_IM-2192-2001.dcm...","[0.148, 0.162, 0.23, 0.054, 0.009, 0.158, 0.5,...",Heart size and mediastinal contour are within ...,Generate a medical report based on the given c...


In [ ]:
df = df.loc[df.index.repeat(3)].reset_index(drop=True)

In [ ]:
df.head(9)

,f,image_names,embedding,report,instruction
0,1,"['1_IM-0001-4001.dcm.png', '1_IM-0001-3001.dcm...","[0.014, 0.137, 0.119, 0.023, 0.0, 0.018, 0.187...",The cardiac silhouette and mediastinum size ar...,Generate a medical report based on the given c...
1,1,"['1_IM-0001-4001.dcm.png', '1_IM-0001-3001.dcm...","[0.014, 0.137, 0.119, 0.023, 0.0, 0.018, 0.187...",The cardiac silhouette and mediastinum size ar...,Generate a medical report based on the given c...
2,1,"['1_IM-0001-4001.dcm.png', '1_IM-0001-3001.dcm...","[0.014, 0.137, 0.119, 0.023, 0.0, 0.018, 0.187...",The cardiac silhouette and mediastinum size ar...,Generate a medical report based on the given c...
3,2,"['2_IM-0652-1001.dcm.png', '2_IM-0652-2001.dcm...","[0.395, 0.132, 0.346, 0.102, 0.121, 0.17, 0.50...",Borderline cardiomegaly. Midline sternotomy En...,Generate a medical report based on the given c...
4,2,"['2_IM-0652-1001.dcm.png', '2_IM-0652-2001.dcm...","[0.395, 0.132, 0.346, 0.102, 0.121, 0.17, 0.50...",Borderline cardiomegaly. Midline sternotomy En...,Generate a medical report based on the given c...
5,2,"['2_IM-0652-1001.dcm.png', '2_IM-0652-2001.dcm...","[0.395, 0.132, 0.346, 0.102, 0.121, 0.17, 0.50...",Borderline cardiomegaly. Midline sternotomy En...,Generate a medical report based on the given c...
6,4,"['4_IM-2050-1001.dcm.png', '4_IM-2050-2001.dcm...","[0.528, 0.658, 0.598, 0.536, 0.215, 0.505, 0.5...",There are diffuse bilateral interstitial and a...,Generate a medical report based on the given c...
7,4,"['4_IM-2050-1001.dcm.png', '4_IM-2050-2001.dcm...","[0.528, 0.658, 0.598, 0.536, 0.215, 0.505, 0.5...",There are diffuse bilateral interstitial and a...,Generate a medical report based on the given c...
8,4,"['4_IM-2050-1001.dcm.png', '4_IM-2050-2001.dcm...","[0.528, 0.658, 0.598, 0.536, 0.215, 0.505, 0.5...",There are diffuse bilateral interstitial and a...,Generate a medical report based on the given c...


In [ ]:
# import ast
# # Convert the string representation of lists to actual lists
# df["embedding"] = df["embedding"].apply(ast.literal_eval)

# # Keep only the first 18 values in each list
# df["embedding"] = df["embedding"].apply(lambda x: x[:18])

In [ ]:
# type(df["embedding"][0])

list

In [ ]:
# df["embedding"][0]

[0.01379562821239233,
 0.13688361644744873,
 0.1191345825791359,
 0.023318637162446976,
 0.0004854995058849454,
 0.017957404255867004,
 0.187190443277359,
 0.013571636751294136,
 0.0036778301000595093,
 0.10351774096488953,
 0.028825515881180763,
 0.1309620887041092,
 0.06667549908161163,
 0.0031406248454004526,
 0.00022423129121307284,
 0.03656288608908653,
 0.035706810653209686,
 0.024243611842393875]

In [ ]:
df.head()

,f,image_names,embedding,report,instruction
0,1,"['1_IM-0001-4001.dcm.png', '1_IM-0001-3001.dcm...","[0.014, 0.137, 0.119, 0.023, 0.0, 0.018, 0.187...",The cardiac silhouette and mediastinum size ar...,Generate a medical report based on the given c...
1,2,"['2_IM-0652-1001.dcm.png', '2_IM-0652-2001.dcm...","[0.395, 0.132, 0.346, 0.102, 0.121, 0.17, 0.50...",Borderline cardiomegaly. Midline sternotomy En...,Generate a medical report based on the given c...
2,4,"['4_IM-2050-1001.dcm.png', '4_IM-2050-2001.dcm...","[0.528, 0.658, 0.598, 0.536, 0.215, 0.505, 0.5...",There are diffuse bilateral interstitial and a...,Generate a medical report based on the given c...
3,5,"['5_IM-2117-1003002.dcm.png', '5_IM-2117-10040...","[0.53, 0.302, 0.508, 0.037, 0.007, 0.353, 0.51...",The cardiomediastinal silhouette and pulmonary...,Generate a medical report based on the given c...
4,6,"['6_IM-2192-1001.dcm.png', '6_IM-2192-2001.dcm...","[0.148, 0.162, 0.23, 0.054, 0.009, 0.158, 0.5,...",Heart size and mediastinal contour are within ...,Generate a medical report based on the given c...


In [ ]:
image_embeddings = df["embedding"]
captions = df["report"].tolist()
instruction_ = df["instruction"].tolist()

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(image_embeddings, captions):
    instructions = instruction_
    inputs = image_embeddings  # Directly use embeddings as inputs
    outputs = captions
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return texts

formatted_prompts = formatting_prompts_func(image_embeddings, captions)

# Save formatted data to a new DataFrame
df_formatted = pd.DataFrame({
    "formatted_prompt": formatted_prompts
})

In [ ]:
formatted_prompts = df_formatted["formatted_prompt"].tolist()

data_dict = {
    "text": formatted_prompts
}

dataset = Dataset.from_dict(data_dict)

In [ ]:
# Split the dataset into train, validation, and test sets
train_test_split = dataset.train_test_split(test_size=0.3, shuffle=True, seed=42)
test_valid_split = train_test_split["test"].train_test_split(test_size=0.75, shuffle=True, seed=42)
dataset_dict = DatasetDict({
    "train": train_test_split["train"],
    "validation": test_valid_split["train"],
    "test": test_valid_split["test"]
})

# Save dataset_dict to disk as JSON files
os.makedirs("dataset_files", exist_ok=True)
dataset_dict["train"].to_json("dataset_files/train.json")
dataset_dict["validation"].to_json("dataset_files/validation.json")
dataset_dict["test"].to_json("dataset_files/test.json")

# Zip the dataset files
with zipfile.ZipFile("dataset_dict.zip", "w") as zipf:
    for root, _, files in os.walk("dataset_files"):
        for file in files:
            zipf.write(os.path.join(root, file), arcname=file)

print("Dataset saved and zipped as 'dataset_dict.zip'")

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset saved and zipped as 'dataset_dict.zip'


In [ ]:
with zipfile.ZipFile("dataset_dict.zip", "r") as zip_ref:
    zip_ref.extractall("unzipped_dataset")

In [ ]:
# Load the JSON files back into `dataset_dict`
dataset_dict = DatasetDict({
    "train": Dataset.from_json("unzipped_dataset/train.json"),
    "validation": Dataset.from_json("unzipped_dataset/validation.json"),
    "test": Dataset.from_json("unzipped_dataset/test.json")
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, #16
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
# Define EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=10,  # Number of evaluations to wait before stopping if no improvement
    early_stopping_threshold=0.01  # Minimum threshold of improvement
)

In [ ]:

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_dict["train"],
    eval_dataset = dataset_dict["validation"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        warmup_steps = 30,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60, # 2972
        learning_rate = 2e-6, #2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        load_best_model_at_end=True,  # Ensure best model is loaded at the end
        metric_for_best_model="eval_loss",  # Define a metric for the best model
        evaluation_strategy="steps",  # Ensure evaluation strategy matches save strategy
        save_strategy="steps"  # Ensure save strategy matches evaluation strategy
    ),
    callbacks=[early_stopping_callback],
)

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 30,
        num_train_epochs = 10, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/2780 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,780 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 870
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.054700
2,2.011000
3,2.032400
4,1.995700
5,1.906300
6,1.770700
7,1.581300
8,1.432700
9,1.333900
10,1.282700


In [ ]:
model.save_pretrained("my_model") # Local saving
tokenizer.save_pretrained("my_model")

('my_model/tokenizer_config.json',
 'my_model/special_tokens_map.json',
 'my_model/tokenizer.json')

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "my_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Extract losses
train_losses = trainer.state.log_history
train_loss_values = [entry['loss'] for entry in train_losses if 'loss' in entry]
val_loss_values = [entry['eval_loss'] for entry in train_losses if 'eval_loss' in entry]
steps = range(len(train_loss_values))

# Save losses to Excel
loss_data = pd.DataFrame({
    'Steps': steps,
    'Train Loss': train_loss_values,
    'Validation Loss': val_loss_values[:len(train_loss_values)]  # Adjust to same length
})
loss_data.to_excel("training_loss.xlsx", index=False)

# Plot losses
plt.figure(figsize=(10, 5))
plt.plot(steps, train_loss_values, label='Train Loss')
plt.plot(steps, val_loss_values[:len(train_loss_values)], label='Validation Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.grid(True)
plt.savefig("loss_plot.png")
plt.show()

print("Losses saved to Excel file and plot generated.")


ValueError: All arrays must be of the same length

In [ ]:
dataset_dict["test"][2]

{'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate a medical report based on the given chest X-ray embedding.\n\n### Input:\n[0.038, 0.314, 0.527, 0.502, 0.001, 0.047, 0.218, 0.034, 0.266, 0.083, 0.003, 0.17, 0.061, 0.0, 0.011, 0.136, 0.144, 0.023, 0.067, 0.295, 0.534, 0.176, 0.042, 0.501, 0.34, 0.046, 0.505, 0.175, 0.063, 0.307, 0.214, 0.008, 0.016, 0.184, 0.61, 0.508]\n\n### Response:\nThe lungs and pleural spaces show no acute abnormality. Heart size and pulmonary vascularity within normal limits.<|end_of_text|>'}

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Generate a medical report based on the given chest X-ray embedding.", # instruction
        [0.038, 0.314, 0.527, 0.502, 0.001, 0.047, 0.218, 0.034, 0.266, 0.083, 0.003, 0.17, 0.061, 0.0, 0.011, 0.136, 0.144, 0.023, 0.067, 0.295, 0.534, 0.176, 0.042, 0.501, 0.34, 0.046, 0.505, 0.175, 0.063, 0.307, 0.214, 0.008, 0.016, 0.184, 0.61, 0.508], # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate a medical report based on the given chest X-ray embedding.\n\n### Input:\n[0.038, 0.314, 0.527, 0.502, 0.001, 0.047, 0.218, 0.034, 0.266, 0.083, 0.003, 0.17, 0.061, 0.0, 0.011, 0.136, 0.144, 0.023, 0.067, 0.295, 0.534, 0.176, 0.042, 0.501, 0.34, 0.046, 0.505, 0.175, 0.063, 0.307, 0.214, 0.008, 0.016, 0.184, 0.61, 0.508]\n\n### Response:\nThe heart normal size. The mediastinum unremarkable. The lungs are clear.<|end_of_text|>']

In [ ]:
example = dataset_dict["test"][1]
text = example['text']

# Split the text at "### Response:"
split_text = text.split("### Response:")
original_report = split_text[-1].replace("<|end_of_text|>", "").strip()
input_part = split_text[0].split("### Input:")[-1].strip()
instruction = split_text[0].split("### Instruction:")[-1].split("### Input:")[0].strip()

# Output the instruction, input part, and original report
print("Instruction:", instruction)
print("Input Part:", input_part)
print("Original Report:", original_report)

Instruction: Generate a medical report based on the given chest X-ray embedding.
Input Part: [0.319, 0.214, 0.264, 0.049, 0.071, 0.5, 0.508, 0.073, 0.029, 0.157, 0.541, 0.505, 0.392, 0.004, 0.007, 0.175, 0.335, 0.265, 0.556, 0.558, 0.538, 0.503, 0.208, 0.501, 0.517, 0.574, 0.098, 0.507, 0.524, 0.531, 0.552, 0.067, 0.282, 0.536, 0.76, 0.528]
Original Report: Heart size and pulmonary vascularity appear within normal limits. The lungs are free focal airspace disease. pleural effusion pneumothora seen.


In [ ]:
prompt = f"Generate a detailed medical report for the given image embedding.### Input:{input_part}### Response:"

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Initialize lists to store original and generated reports
original_reports = []
generated_reports = []
count = 0

# Loop through the test dataset
for example in dataset_dict["test"]:

    text = example['text']

    split_text = text.split("### Response:")
    original_report = split_text[-1].replace("<|end_of_text|>", "").strip()
    input_part = split_text[0].split("### Input:")[-1].strip()
    instruction = split_text[0].split("### Instruction:")[-1].split("### Input:")[0].strip()

    # prompt = f"Generate a detailed medical report for the given image embedding.### Input:{input_part}### Response:"
    alpaca_prompt.format(
        "Generate a medical report based on the given chest X-ray embedding.", # instruction
        input_part, # input
        "", # output - leave this blank for generation!
    )
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the output with adjusted parameters
    outputs = model.generate(**inputs, max_new_tokens=700, length_penalty=3.0, no_repeat_ngram_size=3, early_stopping=True, use_cache=True)
    generated_report = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract only the report part from the generated text
    generated_report = generated_report.split("### Response:")[-1].strip()

    # Append to lists
    original_reports.append(original_report)
    generated_reports.append(generated_report)

    print("Original: ", original_reports[count])
    print("Generated: ", generated_report)

    count += 1
    print(count)
    if count >= 100:
        break

# Create a DataFrame and save to an Excel file
df = pd.DataFrame({
    'Original Report': original_reports,
    'Generated Report': generated_reports
})
df.to_excel("test_results.xlsx", index=False)

print("Test results saved to Excel file.")


Original:  The cardiomediastinal silhouette within normal limits for size and contour. The lungs are normally inflated without evidence focal airspace disease, pleural effusion, pneumothora. Osseous structures are within normal limits for patient age.
Generated:  Heart size within normal limits, stable mediastinal and hilar contours, right chest tip the superior SVC. Surgical clips overlying the left breast. focal areas consolidation. pleural effusions. Mild degenerative changes the thoracic spine.
1
Original:  Heart size and pulmonary vascularity appear within normal limits. The lungs are free focal airspace disease. pleural effusion pneumothora seen.
Generated:  Heart size and mediastinal contours are within normal limits. There no focal opacity. There are mild degenerative changes the spine. There mild thickening the soft tissues over the left clavicle which appears intact. There any tendinitis the right rotator cuff.
2
Original:  The lungs and pleural spaces show no acute abnormali

In [ ]:
# Remove rows where either column contains the word 'XXXX'
df_cleaned = df[~df['Original Report'].str.contains('XXXX', case=False, na=False) &
                ~df['Generated Report'].str.contains('XXXX', case=False, na=False)]

# Save the cleaned DataFrame to an Excel file
df_cleaned.to_excel('cleaned_reports.xlsx', index=False)

# Display the cleaned DataFrame (optional)
print(df_cleaned)

                                      Original Report  \
0   The cardiomediastinal silhouette within normal...   
1   Heart size and pulmonary vascularity appear wi...   
2   The lungs and pleural spaces show no acute abn...   
3   There minimally displaced fracture the right l...   
4   Calcified left lower lobe granuloma. focal are...   
..                                                ...   
95  Clear lungs. Normal heart. pneumothora. pleura...   
96  The cardiomediastinal silhouette within normal...   
97  The cardiac contours are normal. scarring left...   
98  The cardiomediastinal silhouette within normal...   
99  Cardiac and mediastinal silhouette are unremar...   

                                     Generated Report  
0   Heart size within normal limits, stable medias...  
1   Heart size and mediastinal contours are within...  
2   Heart size within normal limits. The lungs are...  
3   Heart size and mediastinal contours are within...  
4   Heart size within normal limits

In [ ]:
# Download required NLTK data
nltk.download('wordnet')

# Calculate BLEU, ROUGE-L, and METEOR scores
smooth_fn = SmoothingFunction().method1
bleu1_scores, bleu2_scores, bleu3_scores, bleu4_scores = [], [], [], []
rougeL_scores = []  # Define this list before use
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
meteor_scores = []

for orig, gen in zip(df_cleaned['Original Report'], df_cleaned['Generated Report']):
    orig_tokens = orig.split()
    gen_tokens = gen.split()

    bleu1_scores.append(sentence_bleu([orig_tokens], gen_tokens, weights=(1, 0, 0, 0), smoothing_function=smooth_fn))
    bleu2_scores.append(sentence_bleu([orig_tokens], gen_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smooth_fn))
    bleu3_scores.append(sentence_bleu([orig_tokens], gen_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smooth_fn))
    bleu4_scores.append(sentence_bleu([orig_tokens], gen_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smooth_fn))

    rougeL_scores.append(scorer.score(orig, gen)['rougeL'][2])
    meteor_scores.append(meteor_score([orig_tokens], gen_tokens))  # Tokenize the hypothesis

# Average the scores
avg_bleu1 = sum(bleu1_scores) / len(bleu1_scores)
avg_bleu2 = sum(bleu2_scores) / len(bleu2_scores)
avg_bleu3 = sum(bleu3_scores) / len(bleu3_scores)
avg_bleu4 = sum(bleu4_scores) / len(bleu4_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)
avg_meteor = sum(meteor_scores) / len(meteor_scores)

print(f"Average BLEU-1: {avg_bleu1:.4f}")
print(f"Average BLEU-2: {avg_bleu2:.4f}")
print(f"Average BLEU-3: {avg_bleu3:.4f}")
print(f"Average BLEU-4: {avg_bleu4:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")
print(f"Average METEOR: {avg_meteor:.4f}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU-1: 0.1879
Average BLEU-2: 0.0987
Average BLEU-3: 0.0592
Average BLEU-4: 0.0383
Average ROUGE-L: 0.2367
Average METEOR: 0.2029
